In [23]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.blob_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.parsers import OpenAIWhisperParser
from langchain_community.document_loaders.parsers.audio import OpenAIWhisperParserLocal
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from typing import List, Tuple

import os.path
import shutil
import dotenv

dotenv.load_dotenv()

LOCAL = True
SAVE_DIR = os.getenv('SAVE_DIR')
CHROMA_PATH = os.getenv('CHROMA_PATH')
WHISPER_MODEL = os.getenv('WHISPER_MODEL')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [72]:
def load_audio_documents(audios: List[str]) -> List[Document]:
    def combine_documents(document_list: List[Document]) -> Document:
        combined_document = [doc for doc_list in document_list for doc in doc_list]
        return combined_document

    loaders = []
    if LOCAL:
        for audio in audios:
            loader = GenericLoader.from_filesystem(path=audio, parser=OpenAIWhisperParserLocal(lang_model=WHISPER_MODEL))
            loaders.append(loader)

    else:
        for audio in audios:
            loader = GenericLoader.from_filesystem(path=audio, parser=OpenAIWhisperParser(api_key=OPENAI_API_KEY))
            loaders.append(loader)

    documents = [loader.load() for loader in loaders]
    documents = combine_documents(documents)
    print(f"Loaded {len(documents)} documents_loaded")
    return documents

In [4]:
def split_documents(documents: List[Document]) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=100,
    )
    chunks = splitter.split_documents(documents)
    print(f"Number of chunks: {len(chunks)}, for {len(documents)} documents")
    print(f"Split documents done !")
    return chunks

In [5]:
def get_vector_store(**kwargs) -> Tuple[Chroma, List[Document]]:
    documents = []
    if 'docs' in kwargs:
        documents = load_audio_documents(kwargs['docs'])
    elif 'urls' in kwargs:
        documents = load_videos(kwargs['urls'])
    else:
        raise ValueError("No documents or urls provided")

    chunks = split_documents(documents)
    vector_store = Chroma.from_documents(documents=chunks,
                                         embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))
    return vector_store, chunks

In [73]:
audio_file = [
    "../../data/audio/mp3/Comment commencer un discours  Ne plus dire bonjour mais captiver laudience dès votre arrivée.mp3",
    "../../data/audio/mp3/Emmanuel Macron  Nous sommes en guerre contre le coronavirus - Covid-19.mp3",
    "../../data/audio/mp3/Molieres  une cérémonie forte en messages politiques.mp3"
]

In [74]:
docs = load_audio_documents(audio_file)

Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing part ../../data/audio/mp3/Comment commencer un discours  Ne plus dire bonjour mais captiver laudience dès votre arrivée.mp3!
Transcribing part ../../data/audio/mp3/Emmanuel Macron  Nous sommes en guerre contre le coronavirus - Covid-19.mp3!
Transcribing part ../../data/audio/mp3/Molieres  une cérémonie forte en messages politiques.mp3!
Loaded 3 documents_loaded


In [80]:
print(docs[1].page_content)

 Nous sommes en guerre. En guerre sanitaire, certes. Nous nous mettons, n'écontre une armée, n'écontre une autre nation. Mais l'ennemi est là, invisible, inséciable qui progressent. Et cela requiert notre mobilisation générale. Nous sommes en guerre. Tout l'action du gouvernement est du parlement. Toi t'être désormais tournée vers le combat contre l'épidémie. De jour, comme de nuit, rien ne doit nous en divertir. C'est pourquoi j'ai décidé que toutes les réformes en cours se rérispendus à commencer par la réforme des retraites.


In [76]:
chunks = split_documents(docs)

Number of chunks: 3, for 3 documents
Split documents done !


In [77]:
vector_store, chunks = get_vector_store(docs=audio_file)

Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using the following model:  openai/whisper-tiny


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Transcribing part ../../data/audio/mp3/Comment commencer un discours  Ne plus dire bonjour mais captiver laudience dès votre arrivée.mp3!
Transcribing part ../../data/audio/mp3/Emmanuel Macron  Nous sommes en guerre contre le coronavirus - Covid-19.mp3!
Transcribing part ../../data/audio/mp3/Molieres  une cérémonie forte en messages politiques.mp3!
Loaded 3 documents_loaded
Number of chunks: 3, for 3 documents
Split documents done !


In [83]:
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import os.path
import shutil
import dotenv

In [85]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [86]:
retriever = vector_store.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

query = "Quel est le type de guerre confronté par Emmanuel Macron ?"

result = rag_chain_with_source.invoke(query)
print(f"Question : {result['question']}")
print(f"Answer : {result['answer']}")
print(f"Sources : {result['context']}")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


Question : Quel est le type de guerre confronté par Emmanuel Macron ?
Answer : Emmanuel Macron est confronté à une guerre sanitaire contre une épidémie, nécessitant une mobilisation générale. Toutes les réformes en cours ont été suspendues pour se concentrer sur la lutte contre l'épidémie. Il est important de rester concentré sur ce combat, de jour comme de nuit.
Sources : [Document(page_content="Nous sommes en guerre. En guerre sanitaire, certes. Nous nous mettons, n'écontre une armée, n'écontre une autre nation. Mais l'ennemi est là, invisible, inséciable qui progressent. Et cela requiert notre mobilisation générale. Nous sommes en guerre. Tout l'action du gouvernement est du parlement. Toi t'être désormais tournée vers le combat contre l'épidémie. De jour, comme de nuit, rien ne doit nous en divertir. C'est pourquoi j'ai décidé que toutes les réformes en cours se rérispendus à commencer par la réforme des retraites.", metadata={'source': '../../data/audio/mp3/Emmanuel Macron  Nous s